In [1]:
# Importing libraries
import geopandas as gpd
from pyproj import CRS
import requests
import geojson


# Fetching data from WFS using requests, in json format, using bounding box over the helsinki area
r = requests.get("""http://lipas.cc.jyu.fi/geoserver/lipas/ows?service=wfs&version=2.0.0&request=GetFeature&typeNames=lipas:lipas_3110_uimahalli&bbox=361500.0001438780454919,6665250.0001345984637737,403750.0001343561452813,6698000.0001281434670091,EPSG:3067&outputFormat=json""")

# Creating GeoDataFrame from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Data check
data.head()

,geometry,id,nimi_fi,nimi_se,nimi_en,sahkoposti,www,puhelinnumero,koulun_liikuntapaikka,vapaa_kaytto,...,allas2_syvyys_max_m,tulospalvelu_aanentoisto,allas2_pituus_m,liukumaet_lkm,allas2_veden_lampo_c,allas5_leveys_m,allas3_syvyys_max_m,allas1_veden_lampo_c,allas3_leveys_m,allas3_veden_lampo_c
0,POINT (370019.100 6670008.185),505790,Espoonlahden uimahalli,Esbovikens simhall,,liikunta@espoo.fi,http://www.espoo.fi/fi-FI/Kulttuuri_ja_liikunt...,09 8165 7576,True,None,...,0.8,true,20,None,28,None,None,27,None,None
1,POINT (396026.139 6684080.788),505797,Hakunilan uimahalli,None,None,liikuntapalvelut@vantaa.fi,http://www.vantaa.fi/fi/liikunta_ja_ulkoilu/ui...,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,POINT (383103.574 6676904.413),505802,Haagan Uimahalli,Haga Simhall,,syk@syk.fi,http://www.syk.fi/uimahalli,(09) 4774 1827,True,None,...,0.9,,10.5,None,27,None,None,27,None,None
3,POINT (386148.117 6673996.129),505803,Helsingin urheilutalo / UH & FIX Kallio / Uima...,None,None,uhkallio@urheiluhallit.fi,http://www.urheiluhallit.fi/kallio.html,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,POINT (384028.393 6675785.522),505822,Invalidisäätiö Orton / Uimahalli,Invalidstiftelse Orton / Simhall,,etunimi.sukunimi@invalidisaatio.fi,www.orton.fi,09 4748 2705,None,None,...,None,,4.2,None,None,None,None,None,None,None


In [2]:
# Removing unnecessary attributes from lipas_data
lipas_data = data[["geometry","id","nimi_fi","nimi_se","tyyppikoodi","tyyppi_nimi_fi"]]

# Data check
lipas_data.head()

,geometry,id,nimi_fi,nimi_se,tyyppikoodi,tyyppi_nimi_fi
0,POINT (370019.100 6670008.185),505790,Espoonlahden uimahalli,Esbovikens simhall,3110,Uimahalli
1,POINT (396026.139 6684080.788),505797,Hakunilan uimahalli,None,3110,Uimahalli
2,POINT (383103.574 6676904.413),505802,Haagan Uimahalli,Haga Simhall,3110,Uimahalli
3,POINT (386148.117 6673996.129),505803,Helsingin urheilutalo / UH & FIX Kallio / Uima...,None,3110,Uimahalli
4,POINT (384028.393 6675785.522),505822,Invalidisäätiö Orton / Uimahalli,Invalidstiftelse Orton / Simhall,3110,Uimahalli


In [3]:
## Creating a list of YKR_IDs based on the location of sport facilities

# Set filepath for YKR grid and read the data
ykr_fp = r"data/MetropAccess_YKR_grid_EurefFIN.shp"
ykr_grid = gpd.read_file(ykr_fp)

# Defining crs for lipas_data (ykr_grid is already set)
lipas_data.crs = {'init':'epsg:3067'}

# Executing a spatial join to find out YKR-grid cells that have a sport facility inside them
lipas_join = gpd.sjoin(lipas_data, ykr_grid, how="inner", op="within")

# Collecting unique YKR_IDs into a list
ykr_list = lipas_join['YKR_ID'].unique().tolist()

# Data check
print(ykr_list)

[5988299, 5889210, 5942152, 5963657, 5949377, 5944001, 5905767, 5936682, 5940293, 5924048, 5823749, 5983496, 5929444, 5889149, 5898366, 5952977, 5968773, 5925830, 5892871, 5872516, 5947626, 5977007, 5911250, 5942121]
